# 🎓 AcademiaOS - Version Gemini

**Automatisation du développement de la Grounded Theory avec Gemini**

Ce notebook reproduit les fonctionnalités d'[AcademiaOS](https://github.com/thomasuebi/academia-os) en utilisant l'API Gemini au lieu d'OpenAI.

## Fonctionnalités
- 📚 Recherche d'articles académiques via Semantic Scholar
- 📝 Codage qualitatif automatisé (méthode Gioia)
- 🔍 Extraction d'informations structurées
- 🧠 Construction théorique automatisée
- 📊 Visualisation des relations conceptuelles

---

## 1. Installation des dépendances

In [ ]:
!pip install -q google-generativeai
!pip install -q semanticscholar
!pip install -q PyPDF2
!pip install -q pandas
!pip install -q tqdm
!pip install -q numpy
!pip install -q scikit-learn

## 2. Configuration de l'API Gemini

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import json
import re
from typing import List, Dict, Any, Optional, Callable
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Configuration de la clé API
# Option 1: Via les secrets Colab (recommandé)
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    # Option 2: Saisie manuelle
    GOOGLE_API_KEY = input("Entrez votre clé API Gemini: ")

genai.configure(api_key=GOOGLE_API_KEY)

# Configuration du modèle
MODEL_NAME = "gemini-1.5-pro"  # ou "gemini-1.5-flash" pour plus de rapidité
EMBEDDING_MODEL = "models/embedding-001"

print(f"✅ API Gemini configurée avec le modèle: {MODEL_NAME}")

## 3. Service Gemini (équivalent à OpenAIService)

In [ ]:
class GeminiService:
    """Service principal pour les appels à l'API Gemini.
    
    Équivalent du OpenAIService d'AcademiaOS, adapté pour Gemini.
    """
    
    def __init__(self, model_name: str = MODEL_NAME):
        self.model = genai.GenerativeModel(model_name)
        self.generation_config = genai.types.GenerationConfig(
            temperature=0.7,
            top_p=0.95,
            top_k=40,
            max_output_tokens=8192,
        )
    
    def generate(self, prompt: str, temperature: float = 0.7) -> str:
        """Génère une réponse à partir d'un prompt."""
        config = genai.types.GenerationConfig(
            temperature=temperature,
            top_p=0.95,
            max_output_tokens=8192,
        )
        response = self.model.generate_content(prompt, generation_config=config)
        return response.text
    
    def generate_json(self, prompt: str, temperature: float = 0.3) -> Any:
        """Génère une réponse JSON structurée."""
        json_prompt = f"""{prompt}

IMPORTANT: Réponds UNIQUEMENT avec un JSON valide, sans texte avant ou après.
Ne commence pas par ```json et ne termine pas par ```.
"""
        response = self.generate(json_prompt, temperature=temperature)
        
        # Nettoyer la réponse
        cleaned = response.strip()
        if cleaned.startswith('```json'):
            cleaned = cleaned[7:]
        if cleaned.startswith('```'):
            cleaned = cleaned[3:]
        if cleaned.endswith('```'):
            cleaned = cleaned[:-3]
        
        try:
            return json.loads(cleaned.strip())
        except json.JSONDecodeError as e:
            print(f"⚠️ Erreur de parsing JSON: {e}")
            print(f"Réponse brute: {response[:500]}...")
            return None
    
    def stream_generate(self, prompt: str, callback: Callable[[str], None] = None):
        """Génère une réponse en streaming."""
        response = self.model.generate_content(prompt, stream=True)
        full_response = ""
        for chunk in response:
            if chunk.text:
                full_response += chunk.text
                if callback:
                    callback(chunk.text)
        return full_response

# Instance globale
gemini_service = GeminiService()
print("✅ GeminiService initialisé")

## 4. Service d'Embeddings et Ranking

In [ ]:
class EmbeddingService:
    """Service pour les embeddings et le ranking sémantique.
    
    Remplace OpenAIEmbeddings de LangChain par les embeddings Gemini.
    """
    
    def __init__(self, model: str = EMBEDDING_MODEL):
        self.model = model
    
    def embed_text(self, text: str) -> List[float]:
        """Génère un embedding pour un texte."""
        result = genai.embed_content(
            model=self.model,
            content=text,
            task_type="retrieval_document"
        )
        return result['embedding']
    
    def embed_query(self, query: str) -> List[float]:
        """Génère un embedding pour une requête."""
        result = genai.embed_content(
            model=self.model,
            content=query,
            task_type="retrieval_query"
        )
        return result['embedding']
    
    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        """Génère des embeddings pour plusieurs textes."""
        embeddings = []
        for text in tqdm(texts, desc="Génération des embeddings"):
            embeddings.append(self.embed_text(text))
        return embeddings


class RankingService:
    """Service de ranking sémantique des articles.
    
    Équivalent du RankingService d'AcademiaOS.
    """
    
    def __init__(self):
        self.embedding_service = EmbeddingService()
    
    def rank_papers(self, query: str, papers: List[Dict]) -> List[Dict]:
        """Classe les articles par pertinence sémantique."""
        if not query or not papers:
            return papers
        
        # Créer les textes à embedder
        texts = []
        for paper in papers:
            text = f"{paper.get('title', '')} {paper.get('abstract', '')}"
            texts.append(text[:2000])  # Limiter la taille
        
        # Générer les embeddings
        query_embedding = self.embedding_service.embed_query(query)
        paper_embeddings = self.embedding_service.embed_texts(texts)
        
        # Calculer les similarités
        similarities = cosine_similarity(
            [query_embedding], 
            paper_embeddings
        )[0]
        
        # Trier par similarité
        ranked_indices = np.argsort(similarities)[::-1]
        ranked_papers = [papers[i] for i in ranked_indices]
        
        # Ajouter les scores
        for i, idx in enumerate(ranked_indices):
            ranked_papers[i]['similarity_score'] = float(similarities[idx])
        
        return ranked_papers

# Instances globales
embedding_service = EmbeddingService()
ranking_service = RankingService()
print("✅ Services d'Embedding et Ranking initialisés")

## 5. Service de Recherche Académique

In [ ]:
from semanticscholar import SemanticScholar

class SearchService:
    """Service de recherche d'articles académiques via Semantic Scholar."""
    
    def __init__(self):
        self.sch = SemanticScholar()
    
    def search_papers(self, query: str, limit: int = 20) -> List[Dict]:
        """Recherche des articles académiques."""
        try:
            results = self.sch.search_paper(query, limit=limit)
            papers = []
            for paper in results:
                papers.append({
                    'id': paper.paperId,
                    'title': paper.title,
                    'abstract': paper.abstract or '',
                    'year': paper.year,
                    'citation_count': paper.citationCount,
                    'authors': [a.name for a in (paper.authors or [])],
                    'url': paper.url,
                    'venue': getattr(paper, 'venue', ''),
                })
            return papers
        except Exception as e:
            print(f"❌ Erreur de recherche: {e}")
            return []
    
    def get_paper_details(self, paper_id: str) -> Optional[Dict]:
        """Récupère les détails d'un article."""
        try:
            paper = self.sch.get_paper(paper_id)
            return {
                'id': paper.paperId,
                'title': paper.title,
                'abstract': paper.abstract or '',
                'year': paper.year,
                'citation_count': paper.citationCount,
                'authors': [a.name for a in (paper.authors or [])],
                'url': paper.url,
                'tldr': getattr(paper, 'tldr', {}).get('text', '') if hasattr(paper, 'tldr') and paper.tldr else '',
            }
        except Exception as e:
            print(f"❌ Erreur: {e}")
            return None

search_service = SearchService()
print("✅ SearchService initialisé")

## 6. Codage Qualitatif - Méthode Gioia

In [ ]:
class GioiaCodingService:
    """Service de codage qualitatif selon la méthode Gioia.
    
    La méthode Gioia comprend 3 niveaux de codage:
    1. Codes de premier ordre (concepts émergents des données)
    2. Codes de second ordre (thèmes agrégés)
    3. Dimensions agrégées (concepts théoriques)
    """
    
    def __init__(self, gemini_service: GeminiService):
        self.gemini = gemini_service
    
    def initial_coding(self, paper: Dict, remarks: str = "") -> List[str]:
        """Codage de premier ordre d'un article.
        
        Identifie les concepts et thèmes émergents du texte.
        """
        prompt = f"""Tu es un chercheur qualitatif expert utilisant la méthode Gioia.

Analyse le texte suivant et identifie les CODES DE PREMIER ORDRE.
Les codes de premier ordre sont des concepts et thèmes émergents directement issus des données,
exprimés dans le langage proche des données sources.

TITRE: {paper.get('title', 'N/A')}

ABSTRACT: {paper.get('abstract', 'N/A')}

TEXTE COMPLET: {paper.get('fullText', paper.get('abstract', ''))[:8000]}

{f'REMARQUES DU CHERCHEUR: {remarks}' if remarks else ''}

Identifie entre 5 et 15 codes de premier ordre pertinents.
Réponds UNIQUEMENT avec un tableau JSON de strings, par exemple:
["code 1", "code 2", "code 3"]
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, list) else []
    
    def second_order_coding(self, first_order_codes: List[str]) -> Dict[str, List[str]]:
        """Codage de second ordre.
        
        Agrège les codes de premier ordre en thèmes plus abstraits.
        """
        prompt = f"""Tu es un chercheur qualitatif expert utilisant la méthode Gioia.

Voici des CODES DE PREMIER ORDRE issus d'une analyse qualitative:
{json.dumps(first_order_codes, ensure_ascii=False, indent=2)}

Regroupe ces codes en CODES DE SECOND ORDRE (thèmes plus abstraits).
Les codes de second ordre représentent des patterns et thèmes qui émergent
de l'agrégation des codes de premier ordre.

Réponds avec un objet JSON où:
- Les clés sont les codes de second ordre (thèmes)
- Les valeurs sont des tableaux de codes de premier ordre correspondants

Exemple de format:
{{
  "Thème A": ["code 1", "code 2"],
  "Thème B": ["code 3", "code 4", "code 5"]
}}
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, dict) else {}
    
    def aggregate_dimensions(self, second_order_codes: Dict[str, List[str]]) -> Dict[str, List[str]]:
        """Agrégation en dimensions théoriques.
        
        Crée des dimensions théoriques à partir des codes de second ordre.
        """
        prompt = f"""Tu es un chercheur qualitatif expert utilisant la méthode Gioia.

Voici des CODES DE SECOND ORDRE issus d'une analyse qualitative:
{json.dumps(second_order_codes, ensure_ascii=False, indent=2)}

Agrège ces codes en DIMENSIONS AGRÉGÉES (concepts théoriques de haut niveau).
Les dimensions agrégées représentent les construits théoriques fondamentaux
qui émergent de l'analyse.

Réponds avec un objet JSON où:
- Les clés sont les dimensions agrégées
- Les valeurs sont des tableaux de codes de second ordre correspondants

Exemple de format:
{{
  "Dimension Théorique 1": ["Thème A", "Thème B"],
  "Dimension Théorique 2": ["Thème C", "Thème D"]
}}
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, dict) else {}
    
    def full_coding_process(self, papers: List[Dict], remarks: str = "") -> Dict:
        """Exécute le processus complet de codage Gioia."""
        print("📝 Étape 1: Codage de premier ordre...")
        all_first_order = []
        for paper in tqdm(papers, desc="Codage initial"):
            codes = self.initial_coding(paper, remarks)
            paper['initial_codes'] = codes
            all_first_order.extend(codes)
        
        # Dédupliquer
        unique_first_order = list(set(all_first_order))
        print(f"   ✅ {len(unique_first_order)} codes de premier ordre identifiés")
        
        print("\n📝 Étape 2: Codage de second ordre...")
        second_order = self.second_order_coding(unique_first_order)
        print(f"   ✅ {len(second_order)} thèmes de second ordre créés")
        
        print("\n📝 Étape 3: Dimensions agrégées...")
        dimensions = self.aggregate_dimensions(second_order)
        print(f"   ✅ {len(dimensions)} dimensions agrégées créées")
        
        return {
            'papers': papers,
            'first_order_codes': unique_first_order,
            'second_order_codes': second_order,
            'aggregate_dimensions': dimensions
        }

gioia_service = GioiaCodingService(gemini_service)
print("✅ GioiaCodingService initialisé")

## 7. Construction de Modèle Théorique

In [ ]:
class ModelingService:
    """Service de construction de modèle théorique.
    
    Équivalent des fonctions de modeling d'AcademiaOS.
    """
    
    def __init__(self, gemini_service: GeminiService):
        self.gemini = gemini_service
    
    def brainstorm_theories(self, coding_results: Dict, remarks: str = "") -> List[Dict]:
        """Identifie les théories applicables aux données."""
        prompt = f"""Tu es un chercheur expert en théorie organisationnelle et sciences sociales.

Analyse ces résultats de codage qualitatif:

DIMENSIONS AGRÉGÉES:
{json.dumps(coding_results.get('aggregate_dimensions', {}), ensure_ascii=False, indent=2)}

CODES DE SECOND ORDRE:
{json.dumps(coding_results.get('second_order_codes', {}), ensure_ascii=False, indent=2)}

{f'CONTEXTE DE RECHERCHE: {remarks}' if remarks else ''}

Identifie 3 à 5 théories existantes qui pourraient s'appliquer à ces données.
Pour chaque théorie, explique pourquoi elle est pertinente.

Réponds avec un tableau JSON:
[
  {{
    "theory_name": "Nom de la théorie",
    "description": "Brève description",
    "relevance": "Pourquoi cette théorie s'applique",
    "key_concepts": ["concept1", "concept2"]
  }}
]
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, list) else []
    
    def identify_relationships(self, coding_results: Dict) -> List[Dict]:
        """Identifie les relations entre concepts."""
        prompt = f"""Analyse ces dimensions et thèmes issus d'une recherche qualitative:

DIMENSIONS AGRÉGÉES:
{json.dumps(coding_results.get('aggregate_dimensions', {}), ensure_ascii=False, indent=2)}

CODES DE SECOND ORDRE:
{json.dumps(coding_results.get('second_order_codes', {}), ensure_ascii=False, indent=2)}

Identifie les RELATIONS potentielles entre ces concepts.
Pour chaque relation, spécifie:
- Les concepts liés
- Le type de relation (causal, corrélation, modération, médiation)
- Une hypothèse de recherche

Réponds avec un tableau JSON:
[
  {{
    "concept_a": "Premier concept",
    "concept_b": "Second concept",
    "relationship_type": "type de relation",
    "hypothesis": "Hypothèse formulée",
    "direction": "A -> B" ou "A <-> B"
  }}
]
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, list) else []
    
    def construct_model(self, coding_results: Dict, theories: List[Dict], 
                       relationships: List[Dict], remarks: str = "") -> Dict:
        """Construit un modèle théorique intégré."""
        prompt = f"""Tu es un chercheur expert en construction théorique.

À partir des éléments suivants, construis un MODÈLE THÉORIQUE intégré:

DIMENSIONS AGRÉGÉES:
{json.dumps(coding_results.get('aggregate_dimensions', {}), ensure_ascii=False, indent=2)}

THÉORIES APPLICABLES:
{json.dumps(theories, ensure_ascii=False, indent=2)}

RELATIONS IDENTIFIÉES:
{json.dumps(relationships, ensure_ascii=False, indent=2)}

{f'CONTEXTE: {remarks}' if remarks else ''}

Construis un modèle théorique cohérent qui:
1. Intègre les dimensions clés
2. S'appuie sur les théories pertinentes
3. Propose des relations causales testables

Réponds avec un objet JSON:
{{
  "model_name": "Nom du modèle",
  "core_proposition": "Proposition centrale du modèle",
  "constructs": [
    {{
      "name": "Nom du construit",
      "definition": "Définition",
      "type": "independent/dependent/mediator/moderator"
    }}
  ],
  "propositions": [
    "P1: Proposition 1",
    "P2: Proposition 2"
  ],
  "theoretical_contribution": "Contribution théorique",
  "boundary_conditions": ["Condition 1", "Condition 2"]
}}
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, dict) else {}
    
    def critique_model(self, model: Dict) -> Dict:
        """Critique le modèle théorique."""
        prompt = f"""Tu es un reviewer académique expérimenté.

Critique ce modèle théorique de manière constructive:
{json.dumps(model, ensure_ascii=False, indent=2)}

Évalue:
1. La cohérence logique
2. L'ancrage théorique
3. La testabilité des propositions
4. Les limites potentielles
5. Les améliorations suggérées

Réponds avec un objet JSON:
{{
  "overall_assessment": "Évaluation générale",
  "strengths": ["Force 1", "Force 2"],
  "weaknesses": ["Faiblesse 1", "Faiblesse 2"],
  "suggestions": ["Suggestion 1", "Suggestion 2"],
  "score": 7  // Note sur 10
}}
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, dict) else {}
    
    def generate_mermaid_diagram(self, model: Dict) -> str:
        """Génère un diagramme Mermaid du modèle."""
        prompt = f"""Crée un diagramme Mermaid pour visualiser ce modèle théorique:
{json.dumps(model, ensure_ascii=False, indent=2)}

Utilise la syntaxe Mermaid flowchart (graph TD ou graph LR).
Inclus:
- Les construits comme nœuds
- Les relations comme flèches avec labels
- Des couleurs pour distinguer les types de construits

Réponds UNIQUEMENT avec le code Mermaid, sans balises de code.
"""
        response = self.gemini.generate(prompt, temperature=0.3)
        # Nettoyer
        cleaned = response.strip()
        if cleaned.startswith('```mermaid'):
            cleaned = cleaned[10:]
        if cleaned.startswith('```'):
            cleaned = cleaned[3:]
        if cleaned.endswith('```'):
            cleaned = cleaned[:-3]
        return cleaned.strip()

modeling_service = ModelingService(gemini_service)
print("✅ ModelingService initialisé")

## 8. Visualisation

In [ ]:
from IPython.display import display, HTML, Markdown

def display_gioia_structure(coding_results: Dict):
    """Affiche la structure de données Gioia."""
    dimensions = coding_results.get('aggregate_dimensions', {})
    second_order = coding_results.get('second_order_codes', {})
    
    html = "<div style='font-family: Arial, sans-serif;'>"
    html += "<h3>📊 Structure de Codage Gioia</h3>"
    html += "<table style='border-collapse: collapse; width: 100%;'>"
    html += "<tr style='background-color: #f0f0f0;'>"
    html += "<th style='border: 1px solid #ddd; padding: 12px; text-align: left;'>Codes 1er Ordre</th>"
    html += "<th style='border: 1px solid #ddd; padding: 12px; text-align: left;'>Thèmes 2nd Ordre</th>"
    html += "<th style='border: 1px solid #ddd; padding: 12px; text-align: left;'>Dimensions Agrégées</th>"
    html += "</tr>"
    
    for dim_name, themes in dimensions.items():
        first_row = True
        for theme in themes:
            first_codes = second_order.get(theme, [])
            codes_html = "<br>".join([f"• {c}" for c in first_codes[:5]])
            if len(first_codes) > 5:
                codes_html += f"<br>... (+{len(first_codes)-5} autres)"
            
            html += "<tr>"
            html += f"<td style='border: 1px solid #ddd; padding: 8px; font-size: 12px;'>{codes_html}</td>"
            html += f"<td style='border: 1px solid #ddd; padding: 8px;'>{theme}</td>"
            if first_row:
                html += f"<td style='border: 1px solid #ddd; padding: 8px; background-color: #e8f4e8; font-weight: bold;' rowspan='{len(themes)}'>{dim_name}</td>"
                first_row = False
            html += "</tr>"
    
    html += "</table></div>"
    display(HTML(html))

def display_mermaid(mermaid_code: str):
    """Affiche un diagramme Mermaid."""
    html = f"""
    <script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
    <script>mermaid.initialize({{startOnLoad:true}});</script>
    <div class="mermaid">
    {mermaid_code}
    </div>
    """
    display(HTML(html))

def display_model(model: Dict):
    """Affiche le modèle théorique."""
    md = f"""## 🧠 {model.get('model_name', 'Modèle Théorique')}

### Proposition Centrale
{model.get('core_proposition', 'N/A')}

### Construits
"""
    for construct in model.get('constructs', []):
        md += f"- **{construct.get('name')}** ({construct.get('type')}): {construct.get('definition')}\n"
    
    md += "\n### Propositions\n"
    for prop in model.get('propositions', []):
        md += f"- {prop}\n"
    
    md += f"\n### Contribution Théorique\n{model.get('theoretical_contribution', 'N/A')}\n"
    
    if model.get('boundary_conditions'):
        md += "\n### Conditions Limites\n"
        for bc in model.get('boundary_conditions', []):
            md += f"- {bc}\n"
    
    display(Markdown(md))

print("✅ Fonctions de visualisation chargées")

---
# 🚀 Utilisation - Exemple Complet
---

## Étape 1: Recherche d'articles

In [ ]:
# Définir votre sujet de recherche
RESEARCH_TOPIC = "digital transformation organizational change"  # Modifiez selon votre sujet
RESEARCHER_REMARKS = "Je m'intéresse aux facteurs de succès de la transformation digitale dans les grandes entreprises."

# Rechercher des articles
print(f"🔍 Recherche d'articles sur: {RESEARCH_TOPIC}")
papers = search_service.search_papers(RESEARCH_TOPIC, limit=15)
print(f"✅ {len(papers)} articles trouvés")

# Afficher les résultats
df = pd.DataFrame(papers)[['title', 'year', 'citation_count', 'authors']]
df['authors'] = df['authors'].apply(lambda x: ', '.join(x[:2]) + ('...' if len(x) > 2 else ''))
display(df)

## Étape 2: Ranking sémantique (optionnel)

In [ ]:
# Classer les articles par pertinence
print("📊 Ranking des articles par pertinence sémantique...")
ranked_papers = ranking_service.rank_papers(
    query=RESEARCHER_REMARKS,
    papers=papers
)

# Garder les top articles
TOP_N = 10
selected_papers = ranked_papers[:TOP_N]
print(f"\n✅ Top {TOP_N} articles sélectionnés:")
for i, p in enumerate(selected_papers, 1):
    print(f"{i}. [{p.get('similarity_score', 0):.3f}] {p['title'][:70]}...")

## Étape 3: Codage Gioia

In [ ]:
# Exécuter le processus de codage complet
print("🔬 Lancement du codage Gioia...\n")
coding_results = gioia_service.full_coding_process(
    papers=selected_papers,
    remarks=RESEARCHER_REMARKS
)

print("\n" + "="*50)
print("📋 RÉSUMÉ DU CODAGE")
print("="*50)
print(f"Codes de 1er ordre: {len(coding_results['first_order_codes'])}")
print(f"Thèmes de 2nd ordre: {len(coding_results['second_order_codes'])}")
print(f"Dimensions agrégées: {len(coding_results['aggregate_dimensions'])}")

In [ ]:
# Visualiser la structure Gioia
display_gioia_structure(coding_results)

## Étape 4: Construction du modèle théorique

In [ ]:
# Brainstorming de théories applicables
print("🧠 Identification des théories applicables...")
theories = modeling_service.brainstorm_theories(coding_results, RESEARCHER_REMARKS)

print(f"\n✅ {len(theories)} théories identifiées:")
for t in theories:
    print(f"\n📚 {t.get('theory_name')}")
    print(f"   {t.get('relevance', '')[:100]}...")

In [ ]:
# Identification des relations
print("🔗 Identification des relations entre concepts...")
relationships = modeling_service.identify_relationships(coding_results)

print(f"\n✅ {len(relationships)} relations identifiées:")
for r in relationships[:5]:
    print(f"   {r.get('concept_a')} {r.get('direction', '->')} {r.get('concept_b')}")
    print(f"   Type: {r.get('relationship_type')}")

In [ ]:
# Construction du modèle
print("🏗️ Construction du modèle théorique...")
model = modeling_service.construct_model(
    coding_results=coding_results,
    theories=theories,
    relationships=relationships,
    remarks=RESEARCHER_REMARKS
)

# Afficher le modèle
display_model(model)

In [ ]:
# Visualisation du modèle
print("📊 Génération du diagramme...")
mermaid_code = modeling_service.generate_mermaid_diagram(model)
print("\nCode Mermaid généré:")
print(mermaid_code)

# Afficher le diagramme
display_mermaid(mermaid_code)

## Étape 5: Critique du modèle

In [ ]:
# Critique du modèle
print("🔍 Analyse critique du modèle...")
critique = modeling_service.critique_model(model)

print(f"\n📋 CRITIQUE DU MODÈLE (Score: {critique.get('score', 'N/A')}/10)")
print("="*50)
print(f"\n📝 Évaluation: {critique.get('overall_assessment', 'N/A')}")

print("\n✅ Points forts:")
for s in critique.get('strengths', []):
    print(f"   • {s}")

print("\n⚠️ Points faibles:")
for w in critique.get('weaknesses', []):
    print(f"   • {w}")

print("\n💡 Suggestions:")
for sug in critique.get('suggestions', []):
    print(f"   • {sug}")

---
## 💾 Export des résultats

In [ ]:
# Sauvegarder tous les résultats
import json
from datetime import datetime

results = {
    'metadata': {
        'research_topic': RESEARCH_TOPIC,
        'remarks': RESEARCHER_REMARKS,
        'date': datetime.now().isoformat(),
        'model': MODEL_NAME
    },
    'papers': [{'id': p['id'], 'title': p['title'], 'initial_codes': p.get('initial_codes', [])} 
              for p in selected_papers],
    'coding_results': {
        'first_order_codes': coding_results['first_order_codes'],
        'second_order_codes': coding_results['second_order_codes'],
        'aggregate_dimensions': coding_results['aggregate_dimensions']
    },
    'theories': theories,
    'relationships': relationships,
    'model': model,
    'critique': critique,
    'mermaid_diagram': mermaid_code
}

# Sauvegarder en JSON
filename = f"grounded_theory_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"✅ Résultats sauvegardés dans: {filename}")

# Télécharger le fichier (Colab)
try:
    from google.colab import files
    files.download(filename)
except:
    print("(Téléchargement automatique non disponible hors Colab)")

---
## 📖 Notes d'utilisation

### Clé API Gemini
1. Allez sur [Google AI Studio](https://makersuite.google.com/app/apikey)
2. Créez une clé API
3. Dans Colab: `Secrets` (🔑) → Ajoutez `GOOGLE_API_KEY`

### Modèles disponibles
- `gemini-1.5-pro` : Plus précis, recommandé pour l'analyse qualitative
- `gemini-1.5-flash` : Plus rapide, bon pour les tests

### Personnalisation
- Modifiez `RESEARCH_TOPIC` pour votre sujet
- Ajoutez des `RESEARCHER_REMARKS` pour guider l'analyse
- Ajustez le nombre d'articles avec `limit` et `TOP_N`

### Limitations
- Semantic Scholar peut avoir des limites de requêtes
- L'analyse dépend de la qualité des abstracts disponibles
- Pour de meilleurs résultats, ajoutez le texte complet des articles (`fullText`)